In [1]:
%load_ext autoreload
%autoreload 2

# Explore, load and download satellite imagery


SPAI has numerous collections of satellite data, from Sentinel to Landsat, through DEMs or climate data. All of this is in the `spai.data.satellite` module.

First, we can check for available collections.


In [ ]:
from spai.data.satellite import AVAILABLE_COLLECTIONS

AVAILABLE_COLLECTIONS

['sentinel-1-grd',
 'sentinel-1-rtc',
 'sentinel-2-l1c',
 'sentinel-2-l2a',
 'sentinel-3-olci',
 'sentinel-3-slstr',
 'sentinel-5p-ch4',
 'sentinel-5p-co',
 'sentinel-5p-no2',
 'sentinel-5p-o3',
 'sentinel-5p-so2',
 'sentinel-5p-hcho',
 'cop-dem-glo-30',
 'cop-dem-glo-90',
 'esa-worldcover',
 'landsat-8-c2-l2',
 'modis-burned-areas',
 'modis-snow-cover-8',
 'modis-snow-cover-daily',
 'alos-palsar-mosaic']

## Explore


The first thing we can do with SPAI is explore and search if there are images available using the `explore_satellite_imagery` function. The parameters are the following:

- aoi: can be anything related to a geometry or a place: the name of a place, a bounding box, a geometry or a GeoDataFrame.
- datetime: str, List or datetime objects. Can be a single date, such as 2020-01-01, or a time interval, such as (2020-01-01, 2020-01-10) or 2020-01-01/2020-01-10. If not provided, searchs the images available in the last month.
- collection: str. One of the AVAILABLE_COLLECTIONS. If not provided, by default is sentinel-2-l2a.


In [3]:
from spai.data.satellite import explore_satellite_imagery

search = explore_satellite_imagery("Manresa")
search[:3]

[{'id': 'S2A_31TDG_20250811_0_L2A',
  'datetime': '2025-08-11T10:49:39Z',
  'thumbnail': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/31/T/DG/2025/8/S2A_31TDG_20250811_0_L2A/preview.jpg'},
 {'id': 'S2A_31TCG_20250811_0_L2A',
  'datetime': '2025-08-11T10:49:41Z',
  'thumbnail': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/31/T/CG/2025/8/S2A_31TCG_20250811_0_L2A/preview.jpg'},
 {'id': 'S2A_31TDG_20250811_1_L2A',
  'datetime': '2025-08-11T10:49:46Z',
  'thumbnail': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/31/T/DG/2025/8/S2A_31TDG_20250811_1_L2A/preview.jpg'}]

Other args can be given.

- crs: the crs of the data to retrieve.
- resolution: the resolution of the data to retrieve. Must be in the crs units. ex. EPSG4326 is in degrees, EPSG3857 in meters.
- bands: bands to retrieve. By default, each collection has default bands. e.g. Sentinel-2-l2a has 1 to 12 (coastal, blue, green ...), or Sentinel-1-grd has vv and vh.
- Other parameters such as cloud_cover, vegetation_percentage, polarization, etc. Each collection has its own filter parameters.


In [4]:
cloud_cover = 50
search = explore_satellite_imagery(
    "Barcelona", ("2020-01-01", "2020-01-10"), "sentinel-2-l2a", cloud_cover=cloud_cover
)
search[:3]

[{'id': 'S2A_31TDG_20200101_0_L2A',
  'datetime': '2020-01-01T10:59:17Z',
  'thumbnail': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/31/T/DG/2020/1/S2A_31TDG_20200101_0_L2A/thumbnail.jpg',
  'eo:cloud_cover': 13.000735},
 {'id': 'S2A_31TDG_20200101_1_L2A',
  'datetime': '2020-01-01T10:59:17Z',
  'thumbnail': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/31/T/DG/2020/1/S2A_31TDG_20200101_1_L2A/thumbnail.jpg',
  'eo:cloud_cover': 7.417369},
 {'id': 'S2A_31TDF_20200101_0_L2A',
  'datetime': '2020-01-01T10:59:28Z',
  'thumbnail': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/31/T/DF/2020/1/S2A_31TDF_20200101_0_L2A/thumbnail.jpg',
  'eo:cloud_cover': 3.678015}]

## Load


Data can be loaded directly into memory as a `xr.Dataset` using the `load_satellite_imagery` function. What this function does is load satellite imagery into memory, from a given area of ​​interest (aoi) and date. So what it returns is a Dataset of xarray. This function is very interesting for performing operations and applying mathematics on the data without having to download it.


In [5]:
from spai.data.satellite import load_satellite_imagery

bands = ["red", "green", "blue"]
data = load_satellite_imagery("Barcelona", bands=bands)
data

<xarray.Dataset> Size: 20MB
Dimensions:                                  (latitude: 1676, longitude: 1953,
                                              time: 1)
Coordinates: (12/47)
  * latitude                                 (latitude) float64 13kB 41.47 .....
  * longitude                                (longitude) float64 16kB 2.052 ....
    spatial_ref                              int32 4B 4326
  * time                                     (time) datetime64[ns] 8B 2025-09...
    created                                  (time) object 8B '2025-09-03T15:...
    platform                                 (time) object 8B 'sentinel-2b'
    ...                                       ...
    earthsearch:s3_path                      (time) object 8B 's3://sentinel-...
    earthsearch:payload_id                   (time) object 8B 'roda-sentinel2...
    earthsearch:boa_offset_applied           (time) object 8B True
    processing:software                      (time) object 8B {'sentinel2-to-...
    updated                                  (time) object 8B '2025-09-03T15:...
    proj:code                                (time) object 8B 'EPSG:32631'
Data variables:
    red                                      (time, latitude, longitude) uint16 7MB dask.array<chunksize=(1, 1676, 1953), meta=np.ndarray>
    green                                    (time, latitude, longitude) uint16 7MB dask.array<chunksize=(1, 1676, 1953), meta=np.ndarray>
    blue                                     (time, latitude, longitude) uint16 7MB dask.array<chunksize=(1, 1676, 1953), meta=np.ndarray>

We can try other collections, like `sentinel-1-grd`.


In [7]:
from spai.data.satellite import load_satellite_imagery
import geopandas as gpd

# Example with no query and crs
data = load_satellite_imagery(
    "Barcelona",
    ("2020-01-01", "2020-01-10"),
    "sentinel-1-grd",
)
data

<xarray.Dataset> Size: 314MB
Dimensions:                                (latitude: 3350, longitude: 3905,
                                            time: 3)
Coordinates: (12/35)
  * latitude                               (latitude) float64 27kB 41.47 ... ...
  * longitude                              (longitude) float64 31kB 2.052 ......
    spatial_ref                            int32 4B 4326
  * time                                   (time) datetime64[ns] 24B 2020-01-...
    datetime                               (time) object 24B '2020-01-04T17:4...
    platform                               (time) object 24B 'SENTINEL-1A' .....
    ...                                     ...
    sar:pixel_spacing_range                (time) object 24B 10 10 10
    sar:observation_direction              (time) object 24B 'right' ... 'right'
    sar:pixel_spacing_azimuth              (time) object 24B 10 10 10
    sar:looks_equivalent_number            (time) object 24B 4.4 4.4 4.4
    s1:instrument_configuration_ID         (time) object 24B '6' '1' '1'
    sat:platform_international_designator  (time) object 24B '2014-016A' ... ...
Data variables:
    vv                                     (time, latitude, longitude) float32 157MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    vh                                     (time, latitude, longitude) float32 157MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>

## Download


Last, data can be downloaded to a SPAI `Storage`. The parameters are:

- storage: the storage to download the data.
- name: the name of the image or images. If it is a single image, it is the entire name of the image. By default is `<collection>_<date>`.
- clip: if the user wants to clip the data to a geometry.
- other parameters already seen as aoi, datetetime, resolution, query, etc.

> In coming sections we will see how to interact with a storage.


We can use the search we done before to download the images for every date.


In [8]:
from spai.data.satellite import download_satellite_imagery
from spai.storage.LocalStorage import LocalStorage

storage = LocalStorage("data")  # Local storage in the `data` folder

# This will download to the storage several images with the name given
# We can control the full name of the imagegdf = gpd.read_file("map.geojson")
for date in ("2020-01-03", "2020-01-08"):
    download_satellite_imagery(
        storage, "Barcelona", date=date, name=f"image_{date}.tif"
    )

'data' created


/Users/juan/Desktop/spai-new/.venv/lib/python3.12/site-packages/rasterio/warp.py:387: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dest = _reproject(


Or we can pass a single date to download.


In [9]:
# This will download a single image for the date, if exists
date = "2020-01-03"
download_satellite_imagery(storage, "Madrid", date)

No images found for 2020-01-03 in sentinel-2-l2a collection
No data available to download
